In [25]:
import requests
import time
import json
from bs4 import BeautifulSoup


PTT_URL = 'https://www.ptt.cc'


def get_web_page(url):
    resp = requests.get(
        url=url,
        cookies={'over18': '1'} # 18歲認證選項
    )
    # 網站無法連線回復None
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text


def get_articles(dom, date):
    soup = BeautifulSoup(dom, 'lxml')

    # 取得上一頁的連結
    paging_div = soup.find('div', class_ = 'btn-group btn-group-paging')
    prev_url = paging_div.find_all('a')[1]['href']

    articles = []  # 儲存取得的文章資料
    divs = soup.find_all('div', class_ = 'r-ent')
    for d in divs:
        if d.find('div', class_ = 'date').text.strip() == date:  # 發文日期等於今天
            # 取得推文數
            push_count = 0
            push_str = d.find('div', class_ = 'nrec').text
            if push_str:
                try:
                    push_count = int(push_str)  # 轉換字串為數字
                except ValueError:
                    # 若轉換失敗，可能是'爆'或 'X1', 'X2'...
                    # 若不是, 不做任何事，push_count 保持為 0
                    if push_str == '爆':
                        push_count = 99
                    elif push_str.startswith('X'):
                        push_count = -10

            # 取得文章連結及標題
            if d.find('a'):  # 有超連結，表示文章存在，未被刪除
                href = d.find('a')['href']
                title = d.find('a').text
                author = d.find('div', class_ = 'author').text
                articles.append({
                    'title': title,
                    'href': href,
                    'push_count': push_count,
                    'author': author
                })
    return articles, prev_url


if __name__ == '__main__':
    current_page = get_web_page(PTT_URL + '/bbs/Gossiping/index.html')
    if current_page:
        articles = []  # 全部的今日文章
        today = time.strftime("%m/%d").lstrip('0')  # 今天日期, 去掉開頭的 '0' 以符合 PTT 網站格式
        current_articles, prev_url = get_articles(current_page, today)  # 目前頁面的今日文章
        while current_articles:  # 若目前頁面有今日文章則加入 articles，並回到上一頁繼續尋找是否有今日文章
            articles += current_articles
            current_page = get_web_page(PTT_URL + prev_url)
            current_articles, prev_url = get_articles(current_page, today)

        # 儲存或處理文章資訊
        print('今天有', len(articles), '篇文章')
        threshold = 50
        print('熱門文章(> %d 推):' % (threshold))
        for a in articles:
            if int(a['push_count']) > threshold:
                print(a)
                
        # 結果輸出成json
        with open('gossiping.json', 'w', encoding='utf-8') as f:
            json.dump(articles, f, indent=2, sort_keys=True, ensure_ascii=False)

今天有 1240 篇文章
熱門文章(> 50 推):
{'title': '[爆卦] 現場需要的物資', 'href': '/bbs/Gossiping/M.1515287269.A.377.html', 'push_count': 99, 'author': 'q10242'}
{'title': '[爆卦] 加入總統府前靜坐抗議現場小祕方', 'href': '/bbs/Gossiping/M.1515285823.A.CFB.html', 'push_count': 99, 'author': 'Roroanlia'}
{'title': '[問卦] 豆花妹跟雞排妹怎麼選', 'href': '/bbs/Gossiping/M.1515303091.A.C27.html', 'push_count': 52, 'author': 'Nyctalopia'}
{'title': '[新聞] 「國昌永明生病了」洪慈庸：警方連醫生都', 'href': '/bbs/Gossiping/M.1515303167.A.6F8.html', 'push_count': 99, 'author': 'file70042'}
{'title': 'Re: [新聞] 工頭闖入靜坐區嗆時力 黃國昌譏:警方', 'href': '/bbs/Gossiping/M.1515298509.A.F21.html', 'push_count': 99, 'author': 'cerberi'}
{'title': '[新聞] 黃國昌徐永明整夜淋雨抗爭\u3000一早遭民眾嗆', 'href': '/bbs/Gossiping/M.1515293625.A.16A.html', 'push_count': 99, 'author': 'teddy4461'}
{'title': '[爆卦] 交大法學院院長被警察阻擋在外', 'href': '/bbs/Gossiping/M.1515293336.A.772.html', 'push_count': 99, 'author': 'chris80634'}
{'title': '[新聞] 女投訴出貨太慢\u3000被淘寶賣家千里尋仇打趴', 'href': '/bbs/Gossiping/M.1515292524.A.2D8.html', 'p